In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
# https://data.census.gov/cedsci/
df = pd.read_csv(
    "ACSDT1Y2022.B08006_2024-09-08T001306/ACSDT1Y2022.B08006-Data.csv", header=1, index_col="Geography")

data = pd.DataFrame(index=df.index)

data["city"] = df["Geographic Area Name"]
data["city"] = data["city"].apply(lambda x: re.sub(r" [a-z]+(.+)?,", ",", x))
data["city"] = data["city"].apply(lambda x: re.sub(r" [A-Z]+?,", ",", x))
data["city"] = data["city"].apply(lambda x: re.sub(r"\/(.+?),", ",", x))
data["city"] = data["city"].apply(lambda x: re.sub(r"\-(.+?),", ",", x))

data["bicycle"] = df["Estimate!!Total:!!Bicycle"].values
# drop all rows where column bicycle is zero
data.drop(data[data["bicycle"] == 0].index, inplace=True)

print(data.to_string())

In [ ]:
df = pd.read_csv(
    "ACSST1Y2022.S0101_2024-09-08T001234/ACSST1Y2022.S0101-Data.csv", header=1, index_col="Geography")

data["population"] = df.loc[data.index,
                            "Estimate!!Total!!Total population"]
data["population"] = pd.to_numeric(data["population"], errors="coerce")
data.drop(data[data["population"] < 50_000].index, inplace=True)
print(data.to_string())

In [ ]:
data["per_100k"] = data["bicycle"] / data["population"] * 100_000
data.dropna(inplace=True)

chicago = data.loc["1600000US1714000", "per_100k"]

data["score"] = (data["per_100k"]/chicago).apply(np.log2)
data.sort_values(by="score", ascending=False, inplace=True)

In [ ]:
print(data.to_string())
data.to_csv("biking.csv")